# Assignment #4: Sudoku Problem GENERATOR

---


Karim Kanji  <br>
IA-20<br>
13.12.2023<br>

#### Normal non-tabu solution, Tabu solution below non-tabu

In [ ]:
import random

# Function to check if placing a digit at a given position is valid
def is_valid(board, row, col, num):
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[i + start_row][j + start_col] == num:
                return False
    return True

# Recursive function to solve the puzzle and ensure a unique solution
def solve(board, empty_spots):
    if not empty_spots:
        return 1  # One solution found
    row, col = empty_spots[-1]
    for num in range(1, 10):
        if is_valid(board, row, col, num):
            board[row][col] = num
            if solve(board, empty_spots[:-1]) == 1:
                return 1  # Stop at first solution
            board[row][col] = 0
    return 0  # No solution found

# Function to remove cells and ensure the puzzle is still solvable
def remove_cells(board, cells_to_remove):
    attempts = cells_to_remove
    while attempts > 0:
        row, col = random.randint(0, 8), random.randint(0, 8)
        while board[row][col] == 0:
            row, col = random.randint(0, 8), random.randint(0, 8)
        backup = board[row][col]
        board[row][col] = 0

        # Copy the board and collect a list of empty spots
        copy_board = [r[:] for r in board]
        empty_spots = [(r, c) for r in range(9) for c in range(9) if copy_board[r][c] == 0]

        # Check for a unique solution
        if solve(copy_board, empty_spots) != 1:
            board[row][col] = backup  # Restore the last removal
            attempts -= 1
        else:
            cells_to_remove -= 1
            attempts = cells_to_remove  # Reset the attempts for the next cell removal

    return board

# Function to print the Sudoku board
def print_board(board):
    for row in board:
        print(" ".join(str(num) if num != 0 else '.' for num in row))
    print()

# Function to create a completed Sudoku board
def create_complete_board():
    base = 3
    side = base * base
    def pattern(r, c): return (base * (r % base) + r // base + c) % side
    def shuffle(s): return random.sample(s, len(s))
    rBase = range(base)
    rows = [g * base + r for g in shuffle(rBase) for r in shuffle(rBase)]
    cols = [g * base + c for g in shuffle(rBase) for c in shuffle(rBase)]
    nums = shuffle(range(1, base * base + 1))

    # Produce board using randomized baseline pattern
    board = [[nums[pattern(r, c)] for c in cols] for r in rows]

    return board

# Example usage
complete_board = create_complete_board()
print("Complete Board:")
print_board(complete_board)

puzzle = remove_cells(complete_board, 54)
print("Puzzle Generated:")
print_board(puzzle)


Complete Board:
1 9 8 6 4 3 5 2 7
5 2 7 8 1 9 4 3 6
4 3 6 7 5 2 1 9 8
9 7 5 1 3 8 2 6 4
3 8 1 4 2 6 9 7 5
2 6 4 5 9 7 3 8 1
8 5 9 3 6 1 7 4 2
7 4 2 9 8 5 6 1 3
6 1 3 2 7 4 8 5 9

Puzzle Generated:
1 . 8 . . . . . 7
5 . . . 1 9 . . .
. . . . . . 1 . .
. 7 . . 3 . . . .
3 . 1 4 2 6 . 7 .
. . . . . 7 3 . 1
8 . 9 3 . 1 . . .
. . . . . . 6 . 3
. . . 2 . . 8 5 .



#### Tabu solution:

In [8]:
import numpy as np
import random

def initial_solution():
    """ Generate an initial random Sudoku solution that adheres to Sudoku rules. """
    base = 3
    side = base * base

    # pattern for a baseline valid solution
    def pattern(r, c): return (base * (r % base) + r // base + c) % side

    # randomize rows, columns and numbers (of valid base pattern)
    def shuffle(s): return random.sample(s, len(s))

    r_base = range(base)
    rows = [g * base + r for g in shuffle(r_base) for r in shuffle(r_base)]
    cols = [g * base + c for g in shuffle(r_base) for c in shuffle(r_base)]
    nums = shuffle(range(1, base * base + 1))

    # produce board using randomized baseline pattern
    board = [[nums[pattern(r, c)] for c in cols] for r in rows]

    return board

def is_valid_move(board, row, col, num):
    """ Check if placing a num in a given row and column is a valid move in Sudoku. """
    for x in range(9):
        if board[row][x] == num or board[x][col] == num:
            return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            if board[i][j] == num:
                return True
    return False

def neighborhood(board):
    """ Generate a neighborhood of the current solution. """
    neighbors = []
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                for num in range(1, 10):
                    if is_valid_move(board, i, j, num):
                        new_board = [row[:] for row in board]
                        new_board[i][j] = num
                        neighbors.append(new_board)
    return neighbors

def objective_function(board):
    """ Objective function to count the number of empty cells. """
    return sum(row.count(0) for row in board)

def tabu_search():
    """ Perform Tabu Search to generate a Sudoku puzzle. """
    max_iterations = 100
    tabu_list = []
    tabu_list_size = 20

    current_solution = initial_solution()
    best_solution = current_solution
    best_score = objective_function(best_solution)

    for _ in range(max_iterations):
        neighborhood_solutions = neighborhood(current_solution)
        neighborhood_solutions = [sol for sol in neighborhood_solutions if sol not in tabu_list]

        if not neighborhood_solutions:
            break

        current_solution = min(neighborhood_solutions, key=objective_function)
        current_score = objective_function(current_solution)

        if current_score < best_score:
            best_solution = current_solution
            best_score = current_score

        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)

    return best_solution

# Generate a Sudoku puzzle using Tabu Search
sudoku_puzzle = tabu_search()
sudoku_puzzle


[[3, 8, 5, 7, 1, 4, 6, 9, 2],
 [9, 2, 6, 5, 3, 8, 7, 1, 4],
 [1, 4, 7, 6, 9, 2, 5, 3, 8],
 [6, 3, 8, 4, 5, 1, 2, 7, 9],
 [7, 9, 2, 8, 6, 3, 4, 5, 1],
 [5, 1, 4, 2, 7, 9, 8, 6, 3],
 [8, 5, 1, 9, 4, 7, 3, 2, 6],
 [2, 6, 3, 1, 8, 5, 9, 4, 7],
 [4, 7, 9, 3, 2, 6, 1, 8, 5]]

#### Function to make the puzzle for the human to solve:

In [9]:
import random

def create_sudoku_puzzle(complete_grid, num_clues=27):
    """
    Create a Sudoku puzzle from a complete grid by removing numbers while ensuring it remains solvable.

    :param complete_grid: A complete and valid Sudoku grid.
    :param num_clues: Number of clues (filled cells) to be left in the puzzle.
    :return: A Sudoku puzzle with the specified number of clues.
    """
    # Flatten the grid for easy handling
    flat_grid = [cell for row in complete_grid for cell in row]

    # Randomly select indices to keep
    indices_to_keep = set(random.sample(range(81), num_clues))

    # Create the puzzle by clearing cells not in indices_to_keep
    puzzle = [flat_grid[i] if i in indices_to_keep else 0 for i in range(81)]

    # Convert back to a 9x9 grid
    return [puzzle[i:i+9] for i in range(0, 81, 9)]

# Given complete Sudoku grid
# complete_grid = [
#     [8, 2, 7, 4, 1, 9, 6, 5, 3],
#     [6, 5, 3, 2, 8, 7, 1, 4, 9],
#     [1, 4, 9, 5, 6, 3, 8, 2, 7],
#     [9, 6, 4, 8, 3, 5, 7, 1, 2],
#     [7, 1, 2, 6, 9, 4, 3, 8, 5],
#     [3, 8, 5, 1, 7, 2, 9, 6, 4],
#     [4, 3, 6, 7, 5, 8, 2, 9, 1],
#     [5, 7, 8, 9, 2, 1, 4, 3, 6],
#     [2, 9, 1, 3, 4, 6, 5, 7, 8]
# ]
complete_grid=sudoku_puzzle

# Generate a Sudoku puzzle from the complete Sudoku grid
sudoku_puzzle = create_sudoku_puzzle(complete_grid)
sudoku_puzzle


[[0, 0, 0, 7, 1, 0, 0, 0, 2],
 [0, 2, 0, 0, 3, 0, 0, 0, 0],
 [0, 0, 7, 0, 0, 0, 5, 3, 0],
 [0, 0, 8, 4, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 5, 0],
 [0, 1, 0, 2, 0, 0, 0, 6, 0],
 [0, 5, 0, 9, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 8, 5, 9, 4, 0],
 [0, 0, 9, 3, 2, 6, 0, 0, 5]]

#### Function to check wether the puzzle is actually human-solveable

In [10]:
def solve_sudoku(puzzle):
    """
    Solve the given Sudoku puzzle using backtracking.
    :param puzzle: 2D list representing a Sudoku puzzle.
    :return: Boolean indicating whether the puzzle is solvable.
    """
    def find_empty_location(pzl):
        """ Find an empty location in the puzzle (marked as 0). """
        for row in range(9):
            for col in range(9):
                if pzl[row][col] == 0:
                    return row, col
        return None

    def is_safe(pzl, row, col, num):
        """ Check if it's safe to place a num in a given row and column. """
        # Check row
        for x in range(9):
            if pzl[row][x] == num:
                return False

        # Check column
        for x in range(9):
            if pzl[x][col] == num:
                return False

        # Check 3x3 box
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(3):
            for j in range(3):
                if pzl[i + start_row][j + start_col] == num:
                    return False

        return True

    def solve(pzl):
        """ Recursive function to solve the Sudoku puzzle. """
        empty_loc = find_empty_location(pzl)
        if not empty_loc:
            return True  # Puzzle solved

        row, col = empty_loc

        for num in range(1, 10):
            if is_safe(pzl, row, col, num):
                pzl[row][col] = num

                if solve(pzl):
                    return True

                pzl[row][col] = 0  # Backtrack

        return False  # Trigger backtracking

    # Create a copy of the puzzle to solve
    pzl_copy = [row[:] for row in puzzle]
    if solve(pzl_copy):
        return True
    else:
        return False

# Check if the Sudoku puzzle is solvable
is_solvable = solve_sudoku(sudoku_puzzle)
is_solvable

True